## From Desktop folder

In [ ]:
# Importing Selenium, which is the library needed to run most of the code
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import math
import random
import datetime
from time import sleep
from docx import Document

# Finding the driver; the location must be found and inserted here.
driver = webdriver.Firefox(executable_path= "C:\\Users\\PetarLuketina\\Desktop\\LinkedIn-Web-Crawler\\geckodriver.exe")


# Setting the paramaters for the web browser
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.cache.disk.enable", False)
profile.set_preference("browser.cache.memory.enable", False)
profile.set_preference("browser.cache.offline.enable", False)
profile.set_preference("network.http.use-cache", False)
profile.update_preferences()


# URL to crawl
driver.get(url = 'https://www.linkedin.com/')


#Sleep haults the code from running. This allows the web browser to fully load before Selenium starts running
sleep(1)

In [ ]:
# Paramaters
email = ''
password = ''

desired_job = 'Python'
desired_location = 'Cleveland/Akron, Ohio Area'


# List of locations that are too far from home
locations_to_avoid = ['Akron, OH, US',
                     'Kent, OH, US',
                     'Elyria, OH, US',
                     'Middlefield, OH, US',
                     'Westlake, OH, US',
                     'Fairlawn, OH, US', 'Fairlawn, Ohio, United States',
                     'Middleburg Heights, OH, US',
                     'Hudson, OH, US',
                     'Lorain, OH, US',
                     'Oberlin, OH, US',
                     'Strongsville, OH, US',
                     'Uniontown, OH, US',
                     'Wooster, OH, US',
                     'Sandusky, OH, US',
                     'Rittman, OH, US',
                     'North Olmsted, OH, US',
                    ]


# List of employers to avoid
employers_to_avoid = ['OnShift',
                     ]


# List of keywords to avoid (that are used in the job title)
keywords_to_avoid = ['c++', 'c ++', 'c + +',
                     'c#', 'c #',
                     '.net',
                     'senior', 'sr', 'sr.',
                     'hadoop',
                     'java',
                     'sas',
                     'manager', 'management',
                     'lead', 'leader',
                     'security',
                    ]

In [ ]:
# Hitting the 'Sign in' button
sign_in = driver.find_element(by='link text', value='Sign in')
sign_in.click()

sleep(1)

In [ ]:
# Finding the input fields to write username and password
email_entry = driver.find_element(by='css selector', value='#username')
password_entry = driver.find_element(by='css selector', value='#password')


# Typing in the email while randomizing the wait-time between key strokes
for letter in email:
    sleep(random.uniform(.1, .4))
    email_entry.send_keys(letter)

    
# Same deal for the password
for letter in password:
    sleep(random.uniform(.1, .4))
    password_entry.send_keys(letter)

    
# Hitting the enter key
password_entry.send_keys(Keys.RETURN)
sleep(3)

In [ ]:
# Opening the search dropdown
search_bar = driver.find_element(by='css selector', value='input')
search_bar.click()
sleep(1)


# Selecting "Jobs" to search
jobs = driver.find_element(by='link text', value='Jobs')
jobs.click()
sleep(2)

In [ ]:
# Finding the input bar with "Search jobs" inside of it
role_input_element = '//input[@placeholder="Search jobs"]'
role = driver.find_element(by='xpath', value=role_input_element)


# Inputting the job title to search
for letter in desired_job:
    sleep(random.uniform(.1, .35))
    role.send_keys(letter)

    
# Inputting the location
location_input_element = '//input[@placeholder="Search location"]'
location = driver.find_element(by='xpath', value=location_input_element)

for letter in desired_location:
    sleep(random.uniform(.1, .35))
    location.send_keys(letter)
    
location.send_keys(Keys.RETURN)
sleep(2)

In [ ]:
# Filtering for the specifics
filter_button_element = '//*/header/div/div/ul/li[5]/button'
filter_button = driver.find_element(by='xpath', value=filter_button_element)
filter_button.click()

In [ ]:
# Selects the date posted (preferably within the last month)
past_month_element = '.ember-view ul > li:nth-child(3) > label > p'
date_posted = driver.find_element(by='css selector', value=past_month_element)
date_posted.click()

full_time_element = '(//*[@class="ember-view"]/ul/li[1]/label/p/span[1])[4]'
job_type = driver.find_element(by='xpath', value=full_time_element)
job_type.click()
sleep(.5)

entry_level_element = '(//*[@class="ember-view"]/ul/li[2]/label/p/span[1])[9]'
entry_level = driver.find_element(by='xpath', value=entry_level_element)
entry_level.click()
sleep(.5)

associate_level_element = '(//*[@class="ember-view"]/ul/li[3]/label/p/span[1])[9]'
associate_level = driver.find_element(by='xpath', value=associate_level_element)
associate_level.click()
sleep(.5)

In [ ]:
submit_element = '//artdeco-modal-header/div/div[2]/button[2]'
apply_filters = driver.find_element(by='xpath', value=submit_element)
apply_filters.click()
sleep(2)

In [ ]:
sort_by = driver.find_element(by='xpath', value='//*[@id="sort-by-select"]')
sort_by.click()
sleep(.5)
by_date = driver.find_element(by='xpath', value='//*[@id="sort-by-select-options"]/li[2]')
by_date.click()
sleep(2)

In [ ]:
# The job element
job_element = '.jobs-search-results__list > li:nth-child({})'

# The elements inside of the job element. These are relative, as indicated by the '.' in the xml path
title_element = './div/artdeco-entity-lockup/artdeco-entity-lockup-content/h3/a'
location_element = './div/artdeco-entity-lockup/artdeco-entity-lockup-content/h5'
employer_element = './div/artdeco-entity-lockup/artdeco-entity-lockup-content/h4'

# Total results returned from the search
x_results = 'div.jobs-search-two-pane__alerts > div > div'

In [ ]:
# Keeps track of which page of the listings we are on
page_number = [1]

def run_thru_listed_positions():
    
    # Iterates through the 25 listings on a single page
    for i in range(1, 26):
        try:
            job = driver.find_element(by='css selector', value=job_element.format(i))

            # Slowly brings the scroll wheel to the bottom. Only need for the first loop
            if i % 3 == 0:
                job.click()

            evalute_job_listing(job)

        # Runs only if there are no more job postings to iterate through
        except:
            pass

    # Printing the current page then adding a +1 because we are going on the next page.
    print('Page ' + str(page_number[0]) + '/' + str(pages) + ' complete...')
    page_number[0] += 1
    
    # If the total pages is more than the page we are currently on,
    # then we do not (and cannot) go to a next page OR if there are nire than 40 pages (which is the max on LinkedIn)
    if page_number[0] <= pages and page_number[0] != 40:
        next_page_element = '//*[@class="search-results-pagination-section"]/artdeco-pagination/ul/li[{}]'
        next_page = driver.find_element(by='xpath', value=next_page_element.format(page_number[0]))
        next_page.click()
        sleep(2)

In [ ]:
found_desirable_positions = []

def evalute_job_listing(job):
    
    title = job.find_element(by='xpath', value=title_element)
    location = job.find_element(by='xpath', value=location_element)
    employer = job.find_element(by='xpath', value=employer_element)
    
    if location.text not in locations_to_avoid:
        status = True
        
        # Tries to find an employer. Strangely, some listings do not have an employer.
        try:
            if employer.text not in employers_to_avoid:
                status = True
            else:
                return False
        except:
            # Any job listing without a company is automatically dismissed
            return False
        
    else:
        return False

    if status == True:
        # Iterates through the job title and makes sure those words are not in the "key_words_avoided" list
        for keyword in (title.text).split():
            if keyword.lower() not in keywords_to_avoid:
                status = True
            else:
                return False

    # If all the paramaters pass, the listing will be added to our 'found_desirable_positions' list
    if status == True:
        if job.text not in found_desirable_positions:
            url = '\n' + title.get_attribute('href')
            found_desirable_positions.append(job.text + url)
        

In [ ]:
# Getting the document paramaters ready

document = Document()

currentDT = datetime.datetime.now()
formatted_time = currentDT.strftime("%a, %b %d, %Y")
document.add_heading('LinkedIn Jobs for ' + str(formatted_time), 0)

In [ ]:
# We find the number of results. Then [0:-8] takes off the "results" at the end and "int()" turns it into an integer
results = int(driver.find_element(by='css selector', value=x_results).text[0:-8])

# Finding how many pages need to be cycled through by dividing the result integer by 25; there are 25 listings per page
pages = math.ceil(results/25)

# Runs for every page with our desired job listing
for i in range(0, pages):
    run_thru_listed_positions()

for listing in found_desirable_positions:
    print('\n' + listing)
    document.add_paragraph(('\n' + listing), style='List Number')

In [ ]:
document.save('LinkedIn Jobs - ' + str(formatted_time) + '.docx')